In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
import sklearn
import numpy as np
import os
import cv2
from keras.optimizers import SGD, Adam
from keras import applications
from keras.models import Sequential, Model
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image


Using TensorFlow backend.


In [8]:
!pip3 install h5py==2.10.0 

     |████████████████████████████████| 2.9MB 7.5MB/s 
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [5]:
image_dir="/content/drive/MyDrive/Dataset/data"
dir_lst=os.listdir(image_dir)
print(dir_lst)

['without_mask', 'with_mask']


In [6]:
image_dir="/content/drive/MyDrive/Dataset/data"
dir_lst=os.listdir(image_dir)
# Create empty list to store the image and its corresponding labels
ximages=[]
y=[]
label_id=0
class_map={}
for i in reversed(dir_lst):
    class_map_temp={i:label_id}
    class_map.update(class_map_temp)
    for index,item in enumerate(os.listdir(os.path.join(image_dir,i))):
        
        ximages.append(cv2.resize(cv2.imread(os.path.join(image_dir,i,item)),(299,299),interpolation = cv2.INTER_AREA)) # Inception V3 image size 229,229,3
        y.append(label_id)
    label_id+=1
print(class_map)

{'with_mask': 0, 'without_mask': 1}


In [7]:
y_images=np.array(y)
print(len(ximages))

1376


In [8]:


base_model= InceptionV3(weights='imagenet',include_top=True, input_shape=(299,299,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


96116736/96112376 [==============================] - 3s 0us/step


In [9]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [ ]:
base_model_nolast_layer= InceptionV3(weights='imagenet', input_shape=(299,299,3),include_top=False)
base_model_nolast_layer.summary()

In [ ]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [ ]:
model.summary()

In [ ]:
print(ximages)

In [ ]:
import time
start = time.time()
features_array = np.zeros((1376,2048))
# extract features from 'avg_pool' layer from InceptionV3. 2048 parameters
for i, img_path in enumerate(ximages):
    x = image.img_to_array(img_path)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    features = features.reshape(2048,)
    features_array[i,:] = features
print('Running time to import features from InceptionV3 model: %.4f seconds' % (time.time()-start))


Running time to import features from InceptionV3 model: 29.4305 seconds


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import time
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
df = pd.DataFrame(features_array)
# Print the features of first 2 images
print(df.head(2))

In [ ]:
# Split images into train and test
Xtrain, Xtest, ytrain, ytest = train_test_split(features_array, y_images, test_size=0.3)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
w=0.4
x=["Train data","Test data"]
ytrain_lst=ytrain.tolist()
ytest_lst=ytest.tolist()
with_mask=[ytrain_lst.count(0),ytest_lst.count(0)]
without_mask=[ytrain_lst.count(1),ytest_lst.count(1)]
print("Number of training data with mask: {} and without mask is:{}".format(with_mask[0],without_mask[0]))
print("Number of test data with mask: {} and without mask is:{}".format(with_mask[1],without_mask[1]))
print("Number of total training data {} and total test data is:{}".format(len(ytrain_lst),len(ytest_lst)))
bar1=np.arange(len(x))
bar2=[i+w for i in bar1]
plt.bar(bar1,with_mask,w,label="with mask")
plt.bar(bar2,without_mask,w,label="without mask")
plt.xlabel("Types of  data")
plt.ylabel("Number of image")
plt.title("Training and Testing data distribution")
plt.xticks(bar1+w/2,x)
plt.legend()
plt.show()

In [ ]:
# Training and testing svm
start = time.time()
# Train the SVM
svm = SVC()
svm.fit(Xtrain, ytrain)
print('Training time of SVM: %.4f seconds' % (time.time()-start))
preds_svm = svm.predict(Xtest)
preds_svm_train=svm.predict(Xtrain)
# Print the accuracy of Gaussian Naive Bayes
print("The testing accuracy of SVM is : ",accuracy_score(ytest, preds_svm))
print("The training accuracy of SVM is : ",accuracy_score(Xtrain, preds_svm_train))


In [ ]:
# Training and testing Gaussian Naive Bayes
start_gnb = time.time()
gnb = GaussianNB()
gnb.fit(Xtrain, ytrain)
print('Training time of GNB: %.4f seconds' % (time.time()-start_gnb))
preds_gnb = gnb.predict(Xtest)
# Print the accuracy of Gaussian Naive Bayes
print("The testing accuracy of GNB is : ",accuracy_score(ytest, preds_gnb))

In [ ]:
# Training and testing of decision tree
start_tree=time.time()
clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(Xtrain, ytrain)
print('Training time of decision tree: %.4f seconds' % (time.time()-start_tree))

preds_tree=clf_tree.predict(Xtest)
print("The testing accuracy of decision tree is : ",accuracy_score(ytest, preds_tree))

Training time of decision tree: 1.4511 seconds
The testing accuracy of decision tree is :  0.9661016949152542


In [ ]:
print("Classification report of SVM on test data: \n",classification_report(ytest, preds_svm))

In [ ]:
print("Confusion matrix  of SVM on test data: \n",confusion_matrix(ytest, preds_svm))

In [ ]:
print("Classification report of Gaussian Naive Bayes on test data: \n",classification_report(ytest, preds_gnb))

In [ ]:
print("Confusion matrix  of Gaussian Naive Bayes on test data: \n",confusion_matrix(ytest, preds_gnb))

In [ ]:
print("Classification report of decision tree on test data: \n",classification_report(ytest, preds_tree))

In [ ]:
print("Confusion matrix  of decision tree on test data: \n",confusion_matrix(ytest, preds_tree))